# DMTA Master Curve: Time-Temperature Superposition

This notebook demonstrates building a master curve from multi-temperature DMTA data using RheoJAX's Mastercurve transform.

## Learning Objectives
- Generate synthetic multi-temperature E*(ω) data with WLF shifting
- Apply time-temperature superposition (TTS) to create a master curve
- Fit a Generalized Maxwell (Prony series) model to the master curve
- Extract WLF parameters and shift factors

**Estimated Time:** 10 minutes

In [1]:
import gc
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np

if os.path.abspath(os.path.join(os.getcwd(), '../..')) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()

np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

FAST_MODE = os.environ.get('FAST_MODE', '1') == '1'
print(f'FAST_MODE: {FAST_MODE}')

FAST_MODE: True


## 1. Generating Multi-Temperature DMTA Data

We simulate a Zener solid at multiple temperatures. The relaxation time shifts with temperature via the WLF equation:

$$\log(a_T) = \frac{-C_1 (T - T_{ref})}{C_2 + (T - T_{ref})}$$

This means E*(ω, T) ≈ E*(ω·aT, Tref) — the same master curve, just frequency-shifted.

In [2]:
from rheojax.core.data import RheoData

# Zener parameters (G-space)
G_e = 1e5     # Equilibrium modulus (Pa)
G_g = 1e8     # Glassy modulus (Pa)
tau_0 = 0.01  # Relaxation time at Tref (s)
nu = 0.5      # Poisson's ratio (rubber)
factor = 2 * (1 + nu)  # = 3.0

# WLF parameters (typical for amorphous polymer)
T_ref = 273.15 + 25  # 25°C in Kelvin
C1 = 8.86
C2 = 101.6

# Temperatures (Kelvin)
temperatures = [273.15 + T for T in [-10, 0, 10, 25, 40, 60, 80]]
omega_base = np.logspace(-1, 2, 30)

# Generate data at each temperature
datasets = []
print(f'{"T (°C)":>8}  {"log(aT)":>8}  {"tau_eff (s)":>12}')
print('-' * 35)

for T in temperatures:
    # WLF shift factor
    dT = T - T_ref
    log_aT = -C1 * dT / (C2 + dT)
    a_T = 10.0**log_aT
    
    # Effective relaxation time at this temperature
    tau_eff = tau_0 / a_T
    
    # Compute G*(ω) for Zener: G*(ω) = G_e + G_m * (iωτ) / (1 + iωτ)
    G_m = G_g - G_e
    iw_tau = 1j * omega_base * tau_eff
    G_star = G_e + G_m * iw_tau / (1 + iw_tau)
    
    # Convert to E*
    E_star = factor * G_star
    
    # Add 2% noise
    noise_real = 1 + 0.02 * np.random.randn(len(omega_base))
    noise_imag = 1 + 0.02 * np.random.randn(len(omega_base))
    E_star_noisy = np.real(E_star) * noise_real + 1j * np.imag(E_star) * noise_imag
    
    data = RheoData(
        x=omega_base, y=E_star_noisy,
        metadata={'temperature': T, 'deformation_mode': 'tension'},
        validate=False,
    )
    datasets.append(data)
    
    print(f'{T - 273.15:8.0f}  {log_aT:8.3f}  {tau_eff:12.4e}')

  T (°C)   log(aT)   tau_eff (s)
-----------------------------------
     -10     4.656    2.2072e-07
       0     2.892    1.2834e-05
      10     1.535    2.9198e-04
      25    -0.000    1.0000e-02
      40    -1.140    1.3797e-01
      60    -2.270    1.8627e+00
      80    -3.112    1.2935e+01


In [3]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

colors = plt.cm.coolwarm(np.linspace(0, 1, len(datasets)))

for data, color in zip(datasets, colors):
    T_C = data.metadata['temperature'] - 273.15
    E_p = np.real(data.y)
    E_pp = np.imag(data.y)
    ax1.loglog(data.x, E_p, 'o', color=color, ms=4, label=f'{T_C:.0f}°C')
    ax2.loglog(data.x, E_pp, 'o', color=color, ms=4, label=f'{T_C:.0f}°C')

ax1.set_xlabel('ω (rad/s)')
ax1.set_ylabel("E' (Pa)")
ax1.set_title("Storage Modulus E'(ω) at Multiple Temperatures")
ax1.legend(fontsize=8, ncol=2)

ax2.set_xlabel('ω (rad/s)')
ax2.set_ylabel('E" (Pa)')
ax2.set_title('Loss Modulus E"(ω) at Multiple Temperatures')
ax2.legend(fontsize=8, ncol=2)

plt.tight_layout()
plt.close('all')

## 2. Time-Temperature Superposition

The `Mastercurve` transform shifts all frequency sweeps onto a single reference temperature curve.

In [4]:
from rheojax.transforms import Mastercurve

# Create master curve with WLF method
mc = Mastercurve(
    reference_temp=T_ref,
    method='wlf',
    C1=C1,
    C2=C2,
)

master, shifts = mc.create_mastercurve(datasets, merge=True, return_shifts=True)

print(f'Master curve: {len(master.x)} points')
print(f'Frequency range: {master.x.min():.2e} - {master.x.max():.2e} rad/s')
print(f'  = {np.log10(master.x.max()/master.x.min()):.1f} decades')

print(f'\nShift factors:')
for T, aT in sorted(shifts.items()):
    print(f'  T = {T - 273.15:6.1f}°C → log(aT) = {np.log10(aT):+.3f}')

Master curve: 210 points
Frequency range: 7.73e-05 - 4.53e+06 rad/s
  = 10.8 decades

Shift factors:
  T =  -10.0°C → log(aT) = +4.656
  T =    0.0°C → log(aT) = +2.892
  T =   10.0°C → log(aT) = +1.535
  T =   25.0°C → log(aT) = +0.000
  T =   40.0°C → log(aT) = -1.140
  T =   60.0°C → log(aT) = -2.270
  T =   80.0°C → log(aT) = -3.112


In [5]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Color by original temperature
colors_master = plt.cm.coolwarm(np.linspace(0, 1, len(datasets)))
source_temps = master.metadata.get('source_temperatures', None)

if source_temps is not None:
    for i, (data, color) in enumerate(zip(datasets, colors_master)):
        T = data.metadata['temperature']
        mask = np.abs(source_temps - T) < 0.1
        T_C = T - 273.15
        ax1.loglog(master.x[mask], np.real(master.y[mask]), 'o', color=color, ms=4, label=f'{T_C:.0f}°C')
        ax2.loglog(master.x[mask], np.imag(master.y[mask]), 'o', color=color, ms=4, label=f'{T_C:.0f}°C')

ax1.set_xlabel('ω·aT (rad/s)')
ax1.set_ylabel("E' (Pa)")
ax1.set_title(f"Master Curve E'(ω·aT) at Tref = {T_ref - 273.15:.0f}°C")
ax1.legend(fontsize=8, ncol=2)

ax2.set_xlabel('ω·aT (rad/s)')
ax2.set_ylabel('E" (Pa)')
ax2.set_title(f'Master Curve E"(ω·aT) at Tref = {T_ref - 273.15:.0f}°C')
ax2.legend(fontsize=8, ncol=2)

plt.tight_layout()
plt.close('all')

## 3. Generalized Maxwell Model Fit

The `GeneralizedMaxwell` model with `modulus_type='tensile'` natively parameterizes E(t) and E*(ω) using a Prony series — no E→G conversion needed.

In [6]:
from rheojax.models.multimode.generalized_maxwell import GeneralizedMaxwell

n_modes = 5 if FAST_MODE else 10

gmm = GeneralizedMaxwell(n_modes=n_modes, modulus_type='tensile')
gmm.fit(
    master.x, master.y,
    test_mode='oscillation',
    optimization_factor=1.5,
)

# Show fitted Prony terms
print(f'Generalized Maxwell ({gmm._n_modes} active modes):')
print(f'{"Mode":>4}  {"E_k (Pa)":>12}  {"tau_k (s)":>12}')
print('-' * 32)

for k in range(gmm._n_modes):
    prefix = 'E' if gmm._modulus_type == 'tensile' else 'G'
    E_k = gmm.parameters.get_value(f'{prefix}_{k+1}')
    tau_k = gmm.parameters.get_value(f'tau_{k+1}')
    print(f'{k+1:4d}  {E_k:12.4e}  {tau_k:12.4e}')

# Equilibrium modulus
E_inf_key = 'E_inf' if gmm._modulus_type == 'tensile' else 'G_inf'
E_inf = gmm.parameters.get_value(E_inf_key)
print(f'\nE_inf = {E_inf:.4e} Pa')
print(f'True G_e × 3 = {G_e * factor:.4e} Pa')

INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=11 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=1.455381s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=31 | final_cost=2.0116e+18 | elapsed=1.455s | final_gradient_norm=3.7315e+15


INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=11 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=0.344007s


...

INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=9 | final_cost=2.0161e+18 | elapsed=1.371s | final_gradient_norm=4.4977e+12


INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=3 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=0.899427s


INFO:nlsq.least_squares:Convergence reason=Both `ftol` and `xtol` termination conditions are satisfied. | iterations=1 | final_cost=2.0161e+18 | elapsed=0.899s | final_gradient_norm=2.2644e+10


Generalized Maxwell (1 active modes):
Mode      E_k (Pa)     tau_k (s)
--------------------------------
   1    8.4713e-03    1.0000e-06

E_inf = 9.2586e+07 Pa
True G_e × 3 = 3.0000e+05 Pa


In [7]:
E_pred = gmm.predict(master.x, test_mode='oscillation')

# GMM returns (N, 2) array [E', E''] not complex
E_pred_prime = E_pred[:, 0]
E_pred_double_prime = E_pred[:, 1]

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(master.x, np.real(master.y), 'ro', ms=3, alpha=0.4, label="E' data")
ax.loglog(master.x, np.imag(master.y), 'bs', ms=3, alpha=0.4, label='E" data')
ax.loglog(master.x, E_pred_prime, 'r-', lw=2, label="E' fit (GMM)")
ax.loglog(master.x, E_pred_double_prime, 'b-', lw=2, label='E" fit (GMM)')
ax.set_xlabel('ω·aT (rad/s)')
ax.set_ylabel('Modulus (Pa)')
ax.set_title(f'Generalized Maxwell Fit to DMTA Master Curve ({gmm._n_modes} modes)')
ax.legend()
plt.tight_layout()
plt.close('all')

# Compute R² using |E*| = sqrt(E'^2 + E''^2)
E_star_mag_data = np.abs(master.y)
E_star_mag_pred = np.sqrt(E_pred_prime**2 + E_pred_double_prime**2)
residual = E_star_mag_data - E_star_mag_pred
ss_res = np.sum(residual**2)
ss_tot = np.sum((E_star_mag_data - np.mean(E_star_mag_data))**2)
R2 = 1 - ss_res / ss_tot
print(f'R² = {R2:.6f}')

R² = -0.007816


## 4. Fitting Real DMTA Master Curve DataThe `data/` directory contains real polymer DMTA data from the [pyvisco](https://github.com/NREL/pyvisco) project (NREL, MIT License). We load the pre-built frequency-domain master curve and fit it with both a Generalized Maxwell (Prony series) model and a fractional Zener model.

In [8]:
import pandas as pd

# Load real frequency-domain master curve (pyvisco, MIT License)
data_dir = os.path.join(os.path.dirname(os.path.abspath('.')), 'dmta', 'data')
if not os.path.exists(data_dir):
    data_dir = os.path.join('.', 'data')

df_master = pd.read_csv(os.path.join(data_dir, 'freq_user_master.csv'), skiprows=[1])
omega_real = 2 * np.pi * df_master['f'].values      # Hz -> rad/s
E_stor_real = df_master['E_stor'].values * 1e6       # MPa -> Pa
E_loss_real = df_master['E_loss'].values * 1e6       # MPa -> Pa
E_star_real = E_stor_real + 1j * E_loss_real

print(f'Master curve: {len(omega_real)} points')
print(f'Frequency range: {omega_real.min():.2e} - {omega_real.max():.2e} rad/s')
print(f'  = {np.log10(omega_real.max() / omega_real.min()):.1f} decades')

# Visualize E'(omega), E''(omega), tan(delta)
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].loglog(omega_real, E_stor_real, 'ro', ms=3, alpha=0.5)
axes[0].set_xlabel(chr(969) + ' (rad/s)')
axes[0].set_ylabel("E' (Pa)")
axes[0].set_title("Storage Modulus E'")

axes[1].loglog(omega_real, E_loss_real, 'bs', ms=3, alpha=0.5)
axes[1].set_xlabel(chr(969) + ' (rad/s)')
axes[1].set_ylabel('E" (Pa)')
axes[1].set_title('Loss Modulus E"')

tan_delta = E_loss_real / E_stor_real
axes[2].semilogx(omega_real, tan_delta, 'g^', ms=3, alpha=0.5)
axes[2].set_xlabel(chr(969) + ' (rad/s)')
axes[2].set_ylabel('tan ' + chr(948))
axes[2].set_title('Loss Tangent')

plt.tight_layout()
plt.close('all')

print(f"E' range: {E_stor_real.min()/1e6:.0f} - {E_stor_real.max()/1e6:.0f} MPa")
print(f'tan(delta) peak: {tan_delta.max():.3f} at {omega_real[np.argmax(tan_delta)]:.2e} rad/s')

Master curve: 206 points
Frequency range: 6.28e-12 - 6.28e+14 rad/s
  = 26.0 decades
E' range: 89 - 9581 MPa
tan(delta) peak: 0.434 at 6.28e+01 rad/s


In [9]:
# Fit Generalized Maxwell to real master curve
n_modes_real = 5 if FAST_MODE else 10

gmm_real = GeneralizedMaxwell(n_modes=n_modes_real, modulus_type='tensile')

gmm_real.fit(
    omega_real, E_star_real,
    test_mode='oscillation',
    optimization_factor=None,
)

E_pred_real = gmm_real.predict(omega_real, test_mode='oscillation')
E_pred_real_prime = E_pred_real[:, 0]
E_pred_real_double_prime = E_pred_real[:, 1]

# R-squared
ss_res_p = np.sum((E_stor_real - E_pred_real_prime)**2)
ss_tot_p = np.sum((E_stor_real - np.mean(E_stor_real))**2)
ss_res_pp = np.sum((E_loss_real - E_pred_real_double_prime)**2)
ss_tot_pp = np.sum((E_loss_real - np.mean(E_loss_real))**2)
R2_prime = 1 - ss_res_p / ss_tot_p
R2_double = 1 - ss_res_pp / ss_tot_pp
print(f"GMM fit ({gmm_real._n_modes} active modes):")
print(f"  R2(E')  = {R2_prime:.6f}")
print(f'  R2(E\'\') = {R2_double:.6f}')

# Plot fit overlay
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(omega_real, E_stor_real, 'ro', ms=3, alpha=0.4, label="E' data")
ax.loglog(omega_real, E_loss_real, 'bs', ms=3, alpha=0.4, label='E" data')
ax.loglog(omega_real, E_pred_real_prime, 'r-', lw=2, label="E' GMM fit")
ax.loglog(omega_real, E_pred_real_double_prime, 'b-', lw=2, label='E" GMM fit')
ax.set_xlabel(chr(969) + ' (rad/s)')
ax.set_ylabel('Modulus (Pa)')
ax.set_title(f'Generalized Maxwell Fit to Real Master Curve ({gmm_real._n_modes} modes)')
ax.legend()
plt.tight_layout()
plt.close('all')

INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=11 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=0.516773s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=43 | final_cost=5.1732e+19 | elapsed=0.517s | final_gradient_norm=4.1638e+16


GMM fit (5 active modes):
  R2(E')  = 0.934622
  R2(E'') = 0.066867


In [10]:
# Compare fitted Prony terms to pyvisco reference
df_prony_ref = pd.read_csv(os.path.join(data_dir, 'prony_terms_reference.csv'), skiprows=[1])
tau_ref = df_prony_ref['tau_i'].values
E_ref = df_prony_ref['E_i'].values * 1e6  # MPa -> Pa

print(f'Reference Prony: {len(tau_ref)} modes')
print(f'RheoJAX GMM: {gmm_real._n_modes} active modes')

# Extract fitted Prony terms
prefix = 'E' if gmm_real._modulus_type == 'tensile' else 'G'
tau_fit = []
E_fit = []
for k in range(gmm_real._n_modes):
    E_k = gmm_real.parameters.get_value(f'{prefix}_{k+1}')
    tau_k = gmm_real.parameters.get_value(f'tau_{k+1}')
    if E_k > 1e-6:  # Skip negligible modes
        tau_fit.append(tau_k)
        E_fit.append(E_k)

tau_fit = np.array(tau_fit)
E_fit = np.array(E_fit)
E_inf_real = gmm_real.parameters.get_value('E_inf' if prefix == 'E' else 'G_inf')

print(f'\nFitted Prony terms (non-negligible):')
print(f'{"Mode":>4}  {"tau_k (s)":>12}  {"E_k (MPa)":>12}')
for k in range(len(tau_fit)):
    print(f'{k+1:4d}  {tau_fit[k]:12.4e}  {E_fit[k]/1e6:12.2f}')
print(f'E_inf = {E_inf_real/1e6:.2f} MPa')

# Discrete relaxation spectrum bar chart
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(np.log10(tau_ref), E_ref / 1e6, width=0.3, alpha=0.5, label=f'pyvisco reference ({len(tau_ref)} modes)', color='steelblue')
if len(tau_fit) > 0:
    ax.bar(np.log10(tau_fit), E_fit / 1e6, width=0.2, alpha=0.7, label=f'RheoJAX GMM ({len(tau_fit)} modes)', color='coral')
ax.set_xlabel('log10(tau / s)')
ax.set_ylabel('E_k (MPa)')
ax.set_title('Discrete Relaxation Spectrum: Reference vs Fitted')
ax.legend()
plt.tight_layout()
plt.close('all')

Reference Prony: 30 modes
RheoJAX GMM: 5 active modes

Fitted Prony terms (non-negligible):
Mode     tau_k (s)     E_k (MPa)
   1    1.0000e-06       4500.28
   2    5.1753e-05       1178.41
   3    1.0178e-03        772.66
   4    3.0294e-02        408.94
   5    6.0349e+00        181.64
E_inf = 116.01 MPa


In [11]:
# Fit fractional Zener solid-solid (4 parameters) for compact representation
if not FAST_MODE:
    from rheojax.models.fractional.fractional_zener_ss import FractionalZenerSolidSolid
    
    fzss = FractionalZenerSolidSolid()
    fzss.fit(omega_real, E_star_real, test_mode='oscillation', deformation_mode='tension')
    fzss_pred = fzss.predict(omega_real, test_mode='oscillation')
    
    # Handle complex vs real output
    if np.iscomplexobj(fzss_pred):
        fzss_prime = np.real(fzss_pred)
        fzss_double = np.imag(fzss_pred)
    elif fzss_pred.ndim == 2 and fzss_pred.shape[1] == 2:
        fzss_prime = fzss_pred[:, 0]
        fzss_double = fzss_pred[:, 1]
    else:
        fzss_prime = np.real(fzss_pred)
        fzss_double = np.imag(fzss_pred)
    
    ss_res_fzss = np.sum((E_stor_real - fzss_prime)**2) + np.sum((E_loss_real - fzss_double)**2)
    ss_tot_fzss = np.sum((E_stor_real - np.mean(E_stor_real))**2) + np.sum((E_loss_real - np.mean(E_loss_real))**2)
    R2_fzss = 1 - ss_res_fzss / ss_tot_fzss
    
    alpha = None
    for name in fzss.parameters.keys():
        if 'alpha' in name.lower():
            alpha = fzss.parameters.get_value(name)
            break
    
    print(f'FZSS fit (4 parameters):')
    print(f'  R2 (combined) = {R2_fzss:.6f}')
    if alpha is not None:
        print(f'  alpha = {alpha:.4f} (fractional order)')
    print(f'  Parameters: {list(fzss.parameters.keys())}')
else:
    fzss_pred = None
    print('FZSS fit skipped in FAST_MODE')

FZSS fit skipped in FAST_MODE


In [12]:
# Model comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# E' comparison
axes[0].loglog(omega_real, E_stor_real, 'ko', ms=3, alpha=0.3, label='Data')
axes[0].loglog(omega_real, E_pred_real_prime, 'r-', lw=2, label=f"GMM ({gmm_real._n_modes} modes)")
if not FAST_MODE and fzss_pred is not None:
    axes[0].loglog(omega_real, fzss_prime, 'b--', lw=2, label='FZSS (4 params)')
axes[0].set_xlabel(chr(969) + ' (rad/s)')
axes[0].set_ylabel("E' (Pa)")
axes[0].set_title("Storage Modulus Comparison")
axes[0].legend()

# Residuals
gmm_res_prime = (E_stor_real - E_pred_real_prime) / E_stor_real * 100
axes[1].semilogx(omega_real, gmm_res_prime, 'r-', lw=1, alpha=0.7, label='GMM')
if not FAST_MODE and fzss_pred is not None:
    fzss_res_prime = (E_stor_real - fzss_prime) / E_stor_real * 100
    axes[1].semilogx(omega_real, fzss_res_prime, 'b--', lw=1, alpha=0.7, label='FZSS')
axes[1].axhline(0, color='k', ls=':', alpha=0.3)
axes[1].set_xlabel(chr(969) + ' (rad/s)')
axes[1].set_ylabel("Relative residual (%)")
axes[1].set_title("E' Residuals")
axes[1].legend()

plt.tight_layout()
plt.close('all')

# tan(delta) overlay
fig, ax = plt.subplots(figsize=(10, 5))
tan_delta_data = E_loss_real / E_stor_real
tan_delta_gmm = E_pred_real_double_prime / np.maximum(E_pred_real_prime, 1e-10)
ax.semilogx(omega_real, tan_delta_data, 'ko', ms=3, alpha=0.3, label='Data')
ax.semilogx(omega_real, tan_delta_gmm, 'r-', lw=2, label='GMM')
if not FAST_MODE and fzss_pred is not None:
    tan_delta_fzss = fzss_double / np.maximum(fzss_prime, 1e-10)
    ax.semilogx(omega_real, tan_delta_fzss, 'b--', lw=2, label='FZSS')
ax.set_xlabel(chr(969) + ' (rad/s)')
ax.set_ylabel('tan ' + chr(948))
ax.set_title('Loss Tangent Comparison')
ax.legend()
plt.tight_layout()
plt.close('all')

In [13]:
# Load raw multi-temperature data for reference
df_raw = pd.read_csv(
    os.path.join(data_dir, 'freq_user_raw.csv'),
    skiprows=[1],
)
df_raw.columns = df_raw.columns.str.strip()
temperatures_real = sorted(df_raw['T'].unique())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
colors_real = plt.cm.coolwarm(np.linspace(0, 1, len(temperatures_real)))

for i, T_val in enumerate(temperatures_real):
    mask = np.abs(df_raw['T'] - T_val) < 0.5
    subset = df_raw[mask]
    label = f'{T_val:.0f}' + chr(176) + 'C' if i % 3 == 0 else None
    ax1.loglog(subset['f'], subset['E_stor'], 'o', color=colors_real[i], ms=4, label=label)
    ax2.loglog(subset['f'], subset['E_loss'], 'o', color=colors_real[i], ms=4, label=label)

ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel("E' (MPa)")
ax1.set_title(f"Raw multi-temperature E'(f) ({len(temperatures_real)} temperatures)")
ax1.legend(fontsize=7, ncol=2)

ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('E" (MPa)')
ax2.set_title(f'Raw multi-temperature E"(f)')
ax2.legend(fontsize=7, ncol=2)

plt.tight_layout()
plt.close('all')

print(f'{len(df_raw)} points across {len(temperatures_real)} temperatures')
print(f'Temperature range: {min(temperatures_real):.0f} to {max(temperatures_real):.0f} ' + chr(176) + 'C')

210 points across 170 temperatures
Temperature range: -50 to 100 °C


## Key Takeaways- **TTS collapses multi-T data** into a single master curve spanning many frequency decades- **WLF parameters** (C1, C2) encode the temperature dependence of relaxation- **GeneralizedMaxwell** with `modulus_type='tensile'` directly fits E*(omega) using Prony series- **Real polymer data** shows a broad glass transition that requires ~5-30 Prony modes for accurate capture- **Fractional models** (FZSS) provide compact 4-parameter alternatives for broad transitions## Next Steps- `03_dmta_fractional_models.ipynb`: Fractional viscoelastic models with Bayesian UQ- `07_dmta_tts_pipeline.ipynb`: Raw multi-temperature to TTS to fit pipeline

In [14]:
del gmm, mc
gc.collect()
jax.clear_caches()
print('Cleanup complete')

Cleanup complete
